# Customer Analytics: Preparing Data for Modeling

![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv` by Sadam, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

## The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called `ds_jobs_clean` that stores the data in `customer_train.csv` much more efficiently. Specifically, they have set the following requirements:

In [388]:
import pandas as pd

ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs_clean = ds_jobs.copy()

In [389]:
df.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,NaN,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,NaN,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,NaN,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,NaN,8,0


### - Columns containing integers must be stored as 32-bit integers (`int32`).

In [390]:
cols_int64 = ds_jobs_clean.columns[ds_jobs_clean.dtypes == int]
ds_jobs_clean[cols_int64] = ds_jobs_clean[cols_int64].astype('int32')

### - Columns containing floats must be stored as 16-bit floats (`float16`).

In [391]:
cols_float64 = ds_jobs_clean.columns[ds_jobs_clean.dtypes == float]
ds_jobs_clean[cols_float64] = ds_jobs_clean[cols_float64].astype('float16')

### - Columns containing nominal categorical data must be stored as the `category` data type.

In [392]:
cat_cols = ['city','gender','relevant_experience', 'enrolled_university',\
    'education_level','major_discipline','experience','company_size',\
            'company_type','last_new_job']

ds_jobs_clean[cat_cols] = ds_jobs_clean[cat_cols].astype('category')

### - Columns containing ordinal categorical data must be stored as _ordered categories_, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [393]:
ds_jobs_clean['relevant_experience'] = ds_jobs_clean['relevant_experience'].cat.set_categories(
    new_categories=[ 'No relevant experience','Has relevant experience'],
    ordered=True)

In [394]:
ds_jobs_clean['enrolled_university'] = ds_jobs_clean['enrolled_university'].cat.set_categories(
    new_categories=['no_enrollment', 'Part time course','Full time course'],
    ordered=True)

In [395]:
ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].cat.set_categories(
    new_categories=['Primary School','High School','Graduate', 'Masters', 'Phd'],
    ordered=True)

In [396]:
ds_jobs_clean['experience'] = ds_jobs_clean['experience'].cat.set_categories(
    new_categories=['<1']+[str(n) for n in (range(1,21))]+['>20'],
    ordered=True)

In [397]:
ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].cat.set_categories(
    new_categories=['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'],
    ordered=True)

In [398]:
ds_jobs_clean['last_new_job'] = ds_jobs_clean['last_new_job'].cat.set_categories(
    new_categories=['never']+[str(n) for n in (range(1,5))]+['>4'],
    ordered=True)

### - The columns of `ds_jobs_clean` must be in the same order as the original dataset.

In [399]:
ds_jobs_clean = ds_jobs_clean.loc[:, ['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change']]

# checking order of columns
print(ds_jobs.columns)
print()
print(ds_jobs_clean.columns)

Index(['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change'],
      dtype='object')

Index(['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change'],
      dtype='object')


### - The DataFrame should be filtered to only contain students with _10 or more years of experience_ at companies with _at least 1000 employees_, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [400]:
ds_jobs_clean = ds_jobs_clean[
    (ds_jobs_clean['experience']>='10') & (ds_jobs_clean['company_size']>='1000-4999')
]

ds_jobs_clean

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19097,25447,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,57,0
19101,6803,city_16,0.910156,Male,Has relevant experience,no_enrollment,High School,NaN,10,10000+,Pvt Ltd,1,89,0
19103,32932,city_10,0.895020,Male,Has relevant experience,Part time course,Masters,Other,>20,1000-4999,Pvt Ltd,>4,18,0
19128,3365,city_16,0.910156,NaN,Has relevant experience,no_enrollment,Graduate,Humanities,>20,1000-4999,Pvt Ltd,>4,23,0


### - calling `info(`) method on `ds_jobs` and `ds_jobs_clean` after preprocessing it, notice the substantial decrease in memory usage.

In [401]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [402]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1

### Preprocessing reduced memory usage from _~2.0+ MB_ to _140.7 KB_ in `ds_jobs_clean`.